# COMED

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import pymssql

from icap.database.icapdatabase import ICapDatabase

In [4]:
fp = 'icap/database/icapdatabase.json'
conn = ICapDatabase(fp).connect()

In [7]:
from icap.comed.comed import COMED

In [14]:
c = COMED(conn)

In [15]:
c.records_.ix['9590268008']['Comed_PL'] - c.records_.ix['9590268008']['PJM_PL']

Year
2015    77.7960
2016   -46.6128
dtype: float64

In [16]:
c.records_.ix['9590268008']

Type,Comed_PL,PJM_PL,MaxPeakLoad
Year,,,
2015,232.8048,155.0088,232.8048
2016,100.5048,147.1176,147.1176


## Define `get_usage` for PJM and COMED

In [17]:
def get_pjm_cp_usage(conn: pymssql.Connection, premise: str = None) -> pd.DataFrame :
    """Select all records from COMED Hourly
    for PJM coincident peak usage. Filter those values that
    do not possess 5 values per year
    """
    
    # query
    query = """
        select 
            h.PremiseId, 
            CAST(cp.CPYearId -1 as INT) as Year, 
            cp.CPDate as CPDatePJM,
            cp.HourEnding as CPHourEnding,
            cp.HourEnding-1 as ADJCPHourEndingPJM, 
            h.Usage as UsagePJM
        from [HourlyUsage] h
        inner join [CoincidentPeak] cp on
            cp.UtilityId = h.UtilityId and
            cp.CPDate = h.UsageDate and
            cp.HourEnding = h.HourEnding + 1
        where
            h.UtilityId = 'COMED' 
            {prem}
         
        order by
            h.PremiseId, cp.CPDate
        """
    if premise is not None:
        pjm_cp_query = query.format(prem="and PremiseId = '%s'" % premise)
    else:
        pjm_cp_query = query.format(prem="", year="")
    
    # read query
    df = pd.read_sql(pjm_cp_query, conn)
    
    # group by premise
    # create filter for len(usage) != 5
    grp = df.groupby(['PremiseId', 'Year'])['UsagePJM'].agg(
        {'CountPJM': len, 'MeanPJM': np.mean})
    grp.reset_index(inplace=True)
    
    # set `Mean` = np.NaN if `Count` != 5
    missing_data_index = grp[grp['CountPJM'] != 5.0].index
    grp = grp.set_value(missing_data_index, 'MeanPJM', np.nan)
    
    
    
    return pd.merge(df, grp, how='left',
                   on=['PremiseId', 'Year'])

In [20]:
def get_comed_cp_usage(conn: pymssql.Connection, premise: str = None) -> pd.DataFrame :
    """Select all records from COMED Hourly
    for COMED coincident peak usage. Filter those values that
    do not possess 5 values per year
    """
    
    # query
    query = """
        select 
            h.PremiseId, 
            CAST(cp.CPYearId -1 as INT) as Year, 
            cp.CPDate as CPDateCOMED, 
            cp.HourEnding as CPHourEndingCOMED, 
            h.Usage as UsageCOMED,
            ZonalLoad
        from [HourlyUsage] h
        inner join [COMED_CoincidentPeak] cp on
            cp.UtilityId = h.UtilityId and
            cp.CPDate = h.UsageDate and
            cp.HourEnding = h.HourEnding
        where
            h.UtilityId = 'COMED'
            {prem}
        order by
            h.PremiseId, cp.CPDate
        """
    # format query for single premise
    if premise is not None:
        pjm_cp_query = query.format(prem="and PremiseId = '%s'" % premise)
    else:
        pjm_cp_query = query.format(prem="")
    
    # read query
    df = pd.read_sql(pjm_cp_query, conn)
    
    # group by premise
    # create filter for len(usage) != 5
    grp = df.groupby(['PremiseId', 'Year'])['UsageCOMED'].agg(
        {'CountCOMED': len, 'MeanCOMED': np.mean})
    grp.reset_index(inplace=True)
    
    # set `Mean` = np.NaN if `Count` != 5
    missing_data_index = grp[grp['CountCOMED'] != 5.0].index
    grp = grp.set_value(missing_data_index, 'MeanCOMED', np.nan)
    
    
    return pd.merge(df, grp, how='left',
                   on=['PremiseId', 'Year'])

### Import usage data

In [52]:
# import and select year=2015

AcustCPL = get_pjm_cp_usage(conn, premise='9590268008')
#AcustCPL = get_pjm_cp_usage(conn)

In [53]:
# import and select year=2015
AcustPL = get_comed_cp_usage(conn, premise='9590268008')
#AcustPL = get_comed_cp_usage(conn)

## Merge mean value dataframes

In [54]:
"""
Join the COMED and PJM usage values on [PremiseId, Year]
"""

# labels for merge
labels_COMED = ['PremiseId', 'Year', 'MeanCOMED']
labels_PJM = ['PremiseId', 'Year', 'MeanPJM']


# merge and drop duplicated rows (result of join operation)
# [PremiseId, Year, MeanCOMED, MeanPJM]
mg = pd.merge(AcustPL[labels_COMED], AcustCPL[labels_PJM], 
                how='inner',
                on=['PremiseId', 'Year']
             ).drop_duplicates()

### Step 3: `AcustCPL >= AcustPL ? AcustCPL : AcustPL`

In [55]:
"""
If the PJM_CP_Mean >= COMED_CP_Mean
    then icap = PJM
else np.NaN

The np.NaN will be replaced once adjustment factors are calculated
"""

def mean_compare(row: pd.Series):
    """logic applie to each row"""
    if row['MeanCOMED'] <= row['MeanPJM']:
        return row['MeanPJM']
    return np.NaN


# [PremiseId, Year, MeanCOMED, MeanPJM, CPLC]
#mg['CPLC'] = mg.apply(mean_compare, axis=1)
mg['CPLC'] = np.NaN
mg = mg.set_index(['PremiseId', 'Year'])

In [56]:
mg.set_value(('9590268008', 2015), 'MeanCOMED', 176.9195)
mg.set_value(('9590268008', 2015), 'MeanPJM', 101.7614)


MeanCOMED   MeanPJM  CPLC
PremiseId  Year                           
9590268008 2014        NaN       NaN   NaN
           2015   176.9195  101.7614   NaN
           2016   100.5048  147.3984   NaN

### Step 4: adjustment factor - denominator

In [64]:
mg.xs(2015, level=1)

,MeanCOMED,MeanPJM,CPLC
PremiseId,,,
9590268008,176.9195,101.7614,NaN


In [75]:
"""
Compute the denominator of the Adjusment factor.
    WeatherNormalizedPeakLoad - AvgComedCPLoad
"""

# Only calculate those years with valid days
valid_count_mask = AcustPL['CountCOMED'] == 5.0

# [PremiseId, Year, MeanComedCPZonalLoad as Mean]
avg_comed_cp_load = AcustPL[valid_count_mask].groupby(['PremiseId', 'Year']
                                 )['ZonalLoad'].agg({'ZonalMean':np.mean}
                                                   )

#WEATHER_NORM = 20900000 - 18881
PJM_ZONAL_LOAD_AVG = 18881
WEATHER_DELTA = 1121264

print(mg)
cell_B41 = mg.xs(2015, level=1).apply(lambda r: r['MeanCOMED'] - r['MeanPJM'], axis=1).values
print(cell_B41)

cell_D41 = cell_B41 / WEATHER_DELTA * 2019 * 1000
print(cell_D41)

mg['CPLC'] = mg['MeanPJM'] + cell_D41
mg
#avg_comed_cp_load['ADJZonalMean'] = WEATHER_NORM #- avg_comed_cp_load['ZonalMean'] 

                 MeanCOMED   MeanPJM        CPLC
PremiseId  Year                                 
9590268008 2014        NaN       NaN         NaN
           2015   176.9195  101.7614  237.094563
           2016   100.5048  147.3984  282.731563
[ 75.1581]
[ 135.3331632]


MeanCOMED   MeanPJM        CPLC
PremiseId  Year                                 
9590268008 2014        NaN       NaN         NaN
           2015   176.9195  101.7614  237.094563
           2016   100.5048  147.3984  282.731563

### Step 4: adjustment factor - numerator

In [49]:
# Index on [PremiseId, Year] and preserve MeanPJM
cpl_usage_avg = AcustCPL[['PremiseId', 'Year', 'MeanPJM']].set_index(['PremiseId', 'Year'])

# Index on [PremiseId, Year] and preserve MeanCOMED
pl_usage_avg = AcustPL[['PremiseId', 'Year', 'MeanCOMED']].set_index(['PremiseId', 'Year'])


# Merge the usage average for each zone
numerator = pd.merge(cpl_usage_avg, pl_usage_avg, 
         left_index=True, right_index=True
        ).drop_duplicates()


# Compute the difference
numerator['Difference'] = numerator['MeanCOMED'] - numerator['MeanPJM']


# Compute adjustment factor
def ratio_factor(row: pd.Series) -> np.float:
    """Compute the adjustment factor"""
    return row['Difference'] / row['ZonalMean']

# Merge `numerator` and `avg_comed_cp_load` to compute adjustment factors for all premises
tmp = pd.DataFrame(pd.merge(numerator, avg_comed_cp_load,
        left_index=True, right_index=True).apply(ratio_factor ,axis=1
                                                ))
tmp.rename(columns={0:'ADJFactor'}, inplace=True)

## Compute Icap

In [45]:
# Join all values in preperation for icap calculation
final = pd.merge(
    pd.merge(tmp, avg_comed_cp_load,
        left_index=True, right_index=True).drop_duplicates(),
    mg,
    left_index=True, right_index=True)

In [46]:
# Icap logic
def compute_icap(r: pd.Series):
    """Implements final Icap logic"""
    if r['MeanPJM'] >= r['MeanCOMED']:
        return r['MeanPJM']
    
    return r['MeanPJM'] + (r['ADJZonalMean'] * r['ADJFactor'])
    
    
# compute the icap
comed_icap = pd.DataFrame(
    final.apply(compute_icap, axis=1)).rename(columns={0:'ICap'})

In [47]:
comed_icap

ICap
PremiseId  Year              
9590268008 2015  88126.979585
           2016    147.398400

In [38]:
final

ADJFactor     ZonalMean  ADJZonalMean  MeanCOMED   MeanPJM  \
PremiseId  Year                                                               
9590268008 2015   0.004216  19718.464453      20881119   232.8048  149.6808   
           2016  -0.002272  20637.782422      20881119   100.5048  147.3984   

                 CPLC  
PremiseId  Year        
9590268008 2015   NaN  
           2016   NaN

## Compare against historical

In [34]:

historical_query = """
    select 
        PremiseId, Cast(CPYearID as INT) Year,
        CapacityTagValue as Historical
    from [CapacityTagHistorical]
    where
        UtilityId = 'COMED'
    """

historical = pd.read_sql(historical_query, conn).set_index(['PremiseId','Year'])

In [35]:
def one_kw(r: pd.Series):
    if np.abs(r['ICap'] - r['Historical']) <= 1.0:
        return 'true'
    return 'false'

compare = pd.merge(comed_icap, historical,
         left_index=True, right_index=True, how='left')

compare['Variance'] = (compare['Historical'] - compare['ICap']) / compare['Historical']
compare['1KW'] = compare.apply(one_kw, axis=1)

## Output to Excel for analysis

In [ ]:
writer = pd.ExcelWriter('comed.xlsx')
compare.to_excel(writer, 'Recipe')
final.to_excel(writer,'factors')
AcustCPL.to_excel(writer, 'AcustCPL')
AcustPL.to_excel(writer, 'ACustPL')
writer.save()